In [1]:
import pandas as pd
from nltk.tokenize import word_tokenize
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

In [2]:
dataset = pd.read_csv('res2.csv', sep=";")
dataset

,Unnamed: 0,Transcription,Emotion,Sentiment,Emotions,Preprocessed_text
0,0,Давай ты успокоишься для начала. Ты можешь не ...,гнев,негативный сентимент,0,давать успокоиться начало мочь вестись ерунда ...
1,1,"Подожди, подожди, успокойся пожалуйста. Подожд...",гнев,негативный сентимент,0,подождать подождать успокоиться пожалуйста под...
2,2,"Кристин, привет. Ты слышала там самолёт упал в...",страх,негативный сентимент,5,кристин привет слышать самолёт упасть зеландия...
3,3,"Кристин, привет .. слушай .. Я боюсь лететь, т...",страх,негативный сентимент,5,кристин привет .. слушать .. бояться лететь ра...
4,4,"Почему? Подожди... Ну подожди, ну как же, но м...",грусть,негативный сентимент,1,почему подождать ... подождать давно планирова...
...,...,...,...,...,...,...
530,530,Я тут готовила медовик. Чтобы приготовить медо...,нейтраль,нейтральный сентимент,2,готовить медовик приготовить медовик очень вре...
531,531,"Значит, правила игры в дурака. Обычного. 36 ка...",нейтраль,нейтральный сентимент,2,значит правило игра дурак обычный 36 карта кол...
532,532,Так. В Третьяковской галерее продается такая и...,нейтраль,нейтральный сентимент,2,третьяковский галерея продаваться такой игра п...
533,533,"я из города Челябинска. Ничего не умел, ничего...",нейтраль,нейтральный сентимент,2,город челябинск уметь хотеть учиться банкир фи...


In [3]:
tones = pd.read_csv('Dictionary_lemmatization_Rus.csv')
tones

,Word,Sum,Number,Weight
0,аборт,-2.0,2.0,-1.0
1,абортивный,-1.0,1.0,-1.0
2,абракадабра,-1.0,1.0,-1.0
3,абсурд,-2.0,2.0,-1.0
4,абсурдность,-1.0,1.0,-1.0
...,...,...,...,...
19632,удивлённо,1.0,1.0,1.0
19633,поражённо,1.0,1.0,1.0
19634,ошеломлённо,1.0,1.0,1.0
19635,ошарашено,1.0,1.0,1.0


## Тональный словрь

### Функция для вывода статического вектора по тексту

In [4]:
def text_to_tonal_vector(text, tone_dictionary):
    vector = list()
    tokens = word_tokenize(text)
    for t in tokens:
        if t in list(tone_dictionary["Word"]):
            vector.append(tone_dictionary[tone_dictionary["Word"]==t].Weight.max())
        else:
            vector.append(0)
    static_vector = list()
    # положительные слова
    static_vector.append(sum([x for x in vector if x>0]))
    # отрицательные слова
    static_vector.append(sum([x for x in vector if x<0]))
    # средний сентимент
    static_vector.append(sum(vector)/len(vector))
    # максимальное значение
    static_vector.append(max(vector))
    # минимальное значение
    static_vector.append(min(vector))
    return static_vector


### Создаем датафрейм

In [5]:
tonal_vectors = pd.DataFrame(columns=["Text", "Positive", "Negative", "Mean", "Max", "Min", "Sentiment"])
for i in dataset.Preprocessed_text:
    arr = list()
    arr.append(i)
    tonal_vector = text_to_tonal_vector(i, tones)
    for v in tonal_vector:
        arr.append(v)
    arr.append(dataset[dataset["Preprocessed_text"]==i].Sentiment.max())
    arr = np.array(arr).reshape(1,7)
    row = pd.DataFrame(arr, columns=["Text", "Positive", "Negative", "Mean", "Max", "Min", "Sentiment"])
    tonal_vectors = tonal_vectors.append(row, ignore_index=True)  # add the row to the DataFrame

/var/folders/9j/ltkckt2s4zsfxrks574_vgfm0000gn/T/ipykernel_21694/285510522.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tonal_vectors = tonal_vectors.append(row, ignore_index=True)  # add the row to the DataFrame
/var/folders/9j/ltkckt2s4zsfxrks574_vgfm0000gn/T/ipykernel_21694/285510522.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tonal_vectors = tonal_vectors.append(row, ignore_index=True)  # add the row to the DataFrame
/var/folders/9j/ltkckt2s4zsfxrks574_vgfm0000gn/T/ipykernel_21694/285510522.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tonal_vectors = tonal_vectors.append(row, ignore_index=True)  # add the row to the DataFrame
/var/folders/9j/ltkckt2s4zsfxrks574_vgfm0000gn/T/ipykernel_21694/28551

In [10]:
tonal_vectors[:428].to_csv('train.csv', sep=",")
tonal_vectors[428:].to_csv('test.csv', sep=",")

In [6]:
tonal_vectors.to_csv('tonal_vectors.csv', sep=",")

## Мешок слов

### Создаем корпус и векторайзер

In [5]:
vectorizer = CountVectorizer()


In [6]:
corpus = dataset.Preprocessed_text
corpus

0      давать успокоиться начало мочь вестись ерунда ...
1      подождать подождать успокоиться пожалуйста под...
2      кристин привет слышать самолёт упасть зеландия...
3      кристин привет .. слушать .. бояться лететь ра...
4      почему подождать ... подождать давно планирова...
                             ...                        
530    готовить медовик приготовить медовик очень вре...
531    значит правило игра дурак обычный 36 карта кол...
532    третьяковский галерея продаваться такой игра п...
533    город челябинск уметь хотеть учиться банкир фи...
534    звать ленога екатерина 23 год родиться 93-м го...
Name: Preprocessed_text, Length: 535, dtype: object

In [7]:
X = vectorizer.fit_transform(corpus).toarray()
X

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

### Ищем все возможные слова

In [39]:
unique_lemms = vectorizer.get_feature_names_out()
unique_lemms

array(['10', '11', '12', ..., 'яхта', 'яхточка', 'ёлка'], dtype=object)

### Записываем в датфрейм

In [33]:
bag_of_words = pd.DataFrame({f'text {i}': X[i] for i in range(X.shape[0])}, index=unique_lemms).T
bag_of_words.tail()

,10,11,12,15,17,18,20,23,30,36,...,яковливетить,яковливич,ялта,январь,яндекс,ярмарка,ясно,яхта,яхточка,ёлка
text 530,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
text 531,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
text 532,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
text 533,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
text 534,0,0,0,0,0,0,0,1,0,0,...,0,0,0,1,0,0,0,0,0,0


In [35]:
bag_of_words.to_csv('bag_of_words.csv', sep=";", index=False)

## TF-IDF

### Все тоже самое что и с мешком слов

In [36]:
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(corpus).toarray()
unique_lemms = vectorizer.get_feature_names_out()
tfidf = pd.DataFrame({f'text {i}': X[i] for i in range(X.shape[0])}, index=unique_lemms).T
tfidf.tail()

,10,11,12,15,17,18,20,23,30,36,...,яковливетить,яковливич,ялта,январь,яндекс,ярмарка,ясно,яхта,яхточка,ёлка
text 530,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,...,0.000000,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0
text 531,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.101637,...,0.000000,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0
text 532,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,...,0.000000,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0
text 533,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,...,0.126743,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0
text 534,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.147437,0.0,0.000000,...,0.000000,0.0,0.0,0.12694,0.0,0.0,0.0,0.0,0.0,0.0


In [37]:
tfidf.to_csv('tfidf.csv', sep=";", index=False)